In [42]:
import pandas as pd
import time
from api import NYTAPI
import spacy
apikey = 'mbAawCmxXm4rc6GtTKCI6NdbUPFtA7qs'
nyt = NYTAPI(apikey)

# url for recipe
url = 'https://cooking.nytimes.com/recipes/1018055-salted-tahini-chocolate-chip-cookies'
# get the next twenty-five comments after the offset numbered comment
offset = 0

# list of dictionaries, each comment is a dictionary
nyt.get_comments(url=url, offset=offset)['comments']

[{'commentID': 112770246,
  'status': 'approved',
  'commentSequence': 112770246,
  'userID': 65299052,
  'userDisplayName': 'lmk',
  'userLocation': '',
  'userTitle': 'NULL',
  'userURL': 'NULL',
  'picURL': None,
  'commentTitle': '<br\\//>',
  'commentBody': "Yum.  These took much longer than 16 minutes to cook - but I'm also in Denver (5000+ ft. altitude) and used a gluten free flour blend.  Either or both of those could have affected the outcome.",
  'createDate': '1620611772',
  'updateDate': '1620612818',
  'approveDate': '1620612818',
  'recommendations': 0,
  'replyCount': 0,
  'replies': [],
  'editorsSelection': False,
  'parentID': None,
  'parentUserDisplayName': None,
  'depth': 1,
  'commentType': 'comment',
  'trusted': 0,
  'recommendedFlag': 0,
  'permID': '112770246',
  'isAnonymous': False},
 {'commentID': 112663059,
  'status': 'approved',
  'commentSequence': 112663059,
  'userID': 156589259,
  'userDisplayName': 'Sonya',
  'userLocation': '',
  'userTitle': 'NUL

In [43]:
def get_recipe_comments(url):
    # from url get recipe comments in dataframe and recipe name
    rows = []
    
    # collect comments
    i = 0
    coms = [i]
    
    while len(coms) > 0:
        offset = 25 * i
        #gets comments from 25i+1 to 25(i+1), stores as a dictionary in coms
        coms = nyt.get_comments(url=url, offset=offset)['comments']
        #appends comments to rows 
        rows = rows + coms
        
        # need to implement 6 second delay due to NYT constraints
        time.sleep(6)
        i += 1
    #create dataframe containing comments.
    df = pd.DataFrame(rows)
    
    # get meta-data from url
    try:
        # splits the url string to extract the recipe id and title.
        ## E.g. https://cooking.nytimes.com/recipes/1018055-salted-tahini-chocolate-chip-cookies 
        ## would return recipe_name = salted tahini chocolate chip cookies
        ## recipe_id = 1018055
        other, recipe = url.split("-")[0], url.split("-")[1:]
        recipe_name = " ".join(recipe)
        recipe_id = other.split("/")[-1]
        df['recipe_id'] = recipe_id
        df['recipe_name'] = recipe_name
    except:
        print("Invalid URL")
        
    return df, recipe_name

In [44]:
df, recipe_name = get_recipe_comments(url)
df

,commentID,status,commentSequence,userID,userDisplayName,userLocation,userTitle,userURL,picURL,commentTitle,...,parentID,parentUserDisplayName,depth,commentType,trusted,recommendedFlag,permID,isAnonymous,recipe_id,recipe_name
0,112770246,approved,112770246,65299052,lmk,,NULL,NULL,None,<br\//>,...,None,None,1,comment,0,0,112770246,False,1018055,salted tahini chocolate chip cookies
1,112663059,approved,112663059,156589259,Sonya,,NULL,NULL,None,<br\//>,...,None,None,1,comment,0,0,112663059,False,1018055,salted tahini chocolate chip cookies
2,112649902,approved,112649902,80047267,KV,,NULL,NULL,None,<br\//>,...,None,None,1,comment,0,0,112649902,False,1018055,salted tahini chocolate chip cookies
3,112368537,approved,112368537,94891199,MaryN,,NULL,NULL,None,<br\//>,...,None,None,1,comment,0,0,112368537,False,1018055,salted tahini chocolate chip cookies
4,112349105,approved,112349105,55886520,Maggie B,,NULL,NULL,None,<br\//>,...,None,None,1,comment,0,0,112349105,False,1018055,salted tahini chocolate chip cookies
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,18199646,approved,18199646,59946471,Stephanie Bellows,<br/>,NULL,NULL,None,<br\//>,...,None,None,1,comment,0,0,18199646,False,1018055,salted tahini chocolate chip cookies
352,18191915,approved,18191915,65219056,Amanda,<br/>,NULL,NULL,None,<br\//>,...,None,None,1,comment,0,0,18191915,False,1018055,salted tahini chocolate chip cookies
353,18192988,approved,18192988,22362535,Audrey,<br/>,NULL,NULL,None,<br\//>,...,None,None,1,comment,0,0,18192988,False,1018055,salted tahini chocolate chip cookies
354,18178053,approved,18178053,71808704,C PAULL,<br/>,NULL,NULL,None,<br\//>,...,None,None,1,comment,0,0,18178053,False,1018055,salted tahini chocolate chip cookies


In [45]:
from bs4 import BeautifulSoup, SoupStrainer
import requests

def get_urls(url):
    # get all urls on a given webpage using Beautiful Soup
    # for from here https://stackoverflow.com/questions/34610162/extract-all-links-from-a-web-page-using-python
    all_urls = []
    page = requests.get(url)    
    data = page.text
    soup = BeautifulSoup(data)
    
    for link in soup.find_all('a'):
        all_urls.append(link.get('href'))
    return all_urls

def only_recipe_urls(list_of_urls):
    # clean a list of urls to only contain recipe urls
    recipe_urls = []
    for url in list_of_urls:
        if type(url) == str:
            if len(url.split("/")) > 1:
                if url.split("/")[1] == 'recipes':
                    recipe_urls.append(url)
    recipe_urls_no_duplicates = []
    [recipe_urls_no_duplicates.append(x) for x in recipe_urls if x not in recipe_urls_no_duplicates]
    return recipe_urls_no_duplicates

In [ ]:
# all urls over N search pages of NYT cooking recipes
N = 100
urls = []
for i in range(N):
    url = f"https://cooking.nytimes.com/search?q=&page={i}"
    urls = urls + get_urls(url)
    
recipe_urls = only_recipe_urls(urls)

In [55]:
df_comments = df[["userDisplayName", "commentBody"]]

In [56]:
for i in range(len(df_comments)):
    if 'a href' in df_comments.loc[i, 'commentBody']:
        df_comments.drop([i], inplace = True)
    else: 
        df_comments.loc[i, 'commentBody'] = df_comments.loc[i, 'commentBody'].replace("<br/>"," ")  

<ipython-input-56-ef1ed2f0bf62>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_comments.loc[i, 'commentBody'] = df_comments.loc[i, 'commentBody'].replace("<br/>"," ")


In [57]:
df_comments.sample(10)

,userDisplayName,commentBody
246,curtis,The addition of a 1/4 c. finely chopped pecans...
336,Cheryl,"They came out very ""cakey"" using cake meal. I..."
190,Tina,"Fresh out of the oven I hated these, but once ..."
77,Heidi,I've been baking lots of cookies in quarantine...
181,Morgan,Wonderful! Used a full cup of Ghirardelli dar...
95,Lucy,Soooooo goooddd
279,Kev,These cookies are Amazing. The dough tastes li...
348,6by6,Read this in the times yesterday and I already...
5,Suzaan,Handled these like refrigerator cookies. Form...
108,Nicole,Yup. Believe every review. They're awesome & p...


In [58]:
df_comments.to_csv("salted-tahini-chocolate-chip-cookies.csv", index = False)

In [37]:
nlp = spacy.load("en_core_web_sm")
comment = df_comments.loc[345, 'commentBody']

In [41]:
comment_lower = comment.lower()

'this from the <a href="http://chabad.org" title="chabad.org" target="_blank">chabad.org</a> website:<br/>pure baking soda in a new container can be used for baking on passover, along with a permissible flour such as potato starch or matzah meal that is certified kosher for passover. baking powder with kosher for passover certification can be used as well.<br/>rochel chein for <a href="http://chabad.org" title="chabad.org" target="_blank">chabad.org</a><br/><br/>however, as with many issues, tradition is determined by the local community and authorities.'

In [28]:
lemmatized = list()


re ADP
: PUNCT
tahini VERB
< X
br/>sauce VERB
or CCONJ
paste VERB
? PUNCT
? PUNCT
? PUNCT
